In [2]:
%matplotlib inline
import pprint  
import math
import numpy as np
import numpy.random as npr
import pandas as pd
import random
import warnings
import collections as ct
from scipy import stats
import scipy.stats as st
from scipy.stats import norm, rayleigh
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from pylab import plot, show,hist,figure,title
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.metrics import r2_score

/anaconda/envs/py36/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
df = pd.read_csv('NBADraft 1985-2016 Scrape.csv')
df_College = pd.read_csv('NBADraft CollegeData Scrape.csv')
df_VORP = df[['Player ID','NBA VORP']]

In [23]:
#Join NBA VORP To College Stats Data table
df_College.merge(df_VORP,left_on='Player ID', right_on = 'Player ID', how = 'left')
# df_College.join(df_VORP,on='Player ID', how = 'outer', lsuffix = '_College', rsuffix = '_NBA')

,Player Name,Player ID,G,MP,FG,FGA,3P,3PA,FT,FTA,...,PF,PTS,Shooting FG%,Shooting 3P%,Shooting FT%,Per Game MP,Per Game PTS,Per Game TRB,Per Game AST,NBA VORP
0,Ben Simmons,simmobe01,33.0,1151.0,216.0,386.0,1.0,3.0,199.0,297.0,...,92.0,632.0,560.0,333.0,670.0,34.9,19.2,11.8,4.8,NaN
1,Brandon Ingram,ingrabr01,36.0,1246.0,214.0,484.0,80.0,195.0,116.0,170.0,...,74.0,624.0,442.0,410.0,682.0,34.6,17.3,6.8,2.0,-1.1
2,Jaylen Brown,brownja02,34.0,939.0,163.0,378.0,30.0,102.0,142.0,217.0,...,108.0,498.0,431.0,294.0,654.0,27.6,14.6,5.4,2.0,-0.7
3,Dragan Bender,bendedr01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.3
4,Kris Dunn,dunnkr01,95.0,2997.0,424.0,943.0,73.0,206.0,293.0,423.0,...,280.0,1214.0,450.0,354.0,693.0,31.5,12.8,5.1,5.8,-0.1
5,Buddy Hield,hieldbu01,132.0,4182.0,783.0,1748.0,349.0,894.0,376.0,450.0,...,259.0,2291.0,448.0,390.0,836.0,31.7,17.4,5.0,1.9,0.1
6,Jamal Murray,murraja01,36.0,1267.0,244.0,538.0,113.0,277.0,119.0,152.0,...,75.0,720.0,454.0,408.0,783.0,35.2,20.0,5.2,2.2,-0.3
7,Marquese Chriss,chrisma01,34.0,848.0,179.0,338.0,21.0,60.0,89.0,130.0,...,138.0,468.0,530.0,350.0,685.0,24.9,13.8,5.4,0.8,0.2
8,Jakob Poeltl,poeltja01,70.0,1885.0,356.0,541.0,0.0,1.0,219.0,361.0,...,165.0,931.0,658.0,0.0,607.0,26.9,13.3,8.0,1.3,0.2
9,Thon Maker,makerth01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1


In [11]:
#Calculate VORP grouped by Pick #
VORP_Per_Pick_Mean = df['NBA VORP'].groupby([df['Pk']]).mean()
VORP_Per_Pick_High = df['NBA VORP'].groupby([df['Pk']]).max()
d = {'VORP Per Pick High': VORP_Per_Pick_High[0:60],'VORP Per Pick Mean': VORP_Per_Pick_Mean[0:60]}
VORP_Per_Pick = pd.DataFrame(data = d)
print(VORP_Per_Pick)

In [4]:
df_filtered = df
#enc = OneHotEncoder
#df_train['NBA VORP'] = pd.to_numeric(df_train['NBA VORP'], errors='coerce')
one_hot_college = pd.get_dummies(df_filtered['Round 1 College'])
one_hot_college = one_hot_college.apply(pd.to_numeric, errors='ignore')
one_hot_team = pd.get_dummies(df_filtered['Tm'])
one_hot_team = one_hot_team.apply(pd.to_numeric, errors='ignore')
df_filtered = df_filtered.drop('Round 1 College', axis = 1)
df_filtered = df_filtered.drop('Tm', axis = 1)
df_filtered = df_filtered.join(one_hot_college)
df_filtered = df_filtered.join(one_hot_team)
df_temp = df_filtered.iloc[:,5:]
df_filtered = df_filtered.drop(df_filtered.columns[5:],axis = 1)

In [5]:
df_filtered.head()

,Year,Rk,Pk,Round 1 Player,Player ID
0,2016,1,1,Ben Simmons\simmobe01,simmobe01
1,2016,2,2,Brandon Ingram\ingrabr01,ingrabr01
2,2016,3,3,Jaylen Brown\brownja02,brownja02
3,2016,4,4,Dragan Bender\bendedr01,bendedr01
4,2016,5,5,Kris Dunn\dunnkr01,dunnkr01


In [178]:
fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
df_imputed = pd.DataFrame(fill_NaN.fit_transform(df_temp))
df_imputed.columns = df_temp.columns
df_imputed.index = df_temp.index
df_filtered = df_filtered.join(df_imputed)
df_filtered.dropna()
df_query = df_filtered

In [179]:
drop_index = []
for i in range(0,len(df)):
    if math.isnan(df_filtered['NBA VORP'][i]) or math.isnan(df_filtered['NBA VORP Count'][i]) or df_filtered['NBA VORP Count'][i] < 3.0:
        drop_index.append(i)
        
df_filtered = df_filtered.drop(df_filtered.index[drop_index])
df_filtered['is_train'] = np.random.uniform(0, 1, len(df_filtered)) <= .75
train, test = df_filtered[df_filtered['is_train']==True], df_filtered[df_filtered['is_train']==False]
train = train.drop('is_train',axis = 1)
test = test.drop('is_train', axis = 1)
                                                                                             

In [180]:
train.head()

,Year,Rk,Pk,Round 1 Player,Player ID,Yrs,Totals G,Totals MP,Totals PTS,Totals TRB,...,PHO,POR,SAC,SAS,SEA,TOR,UTA,VAN,WAS,WSB
5,2016,6,6,Buddy Hield\hieldbu01,hieldbu01,1.0,82.0,1888.0,866.0,269.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,2015,2,2,D'Angelo Russell\russeda01,russeda01,2.0,143.0,4070.0,2038.0,496.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,2015,3,3,Jahlil Okafor\okafoja01,okafoja01,2.0,103.0,2725.0,1518.0,613.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,2015,4,4,Kristaps Porzingis\porzikr01,porzikr01,2.0,138.0,4211.0,2224.0,1001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,2015,5,5,Mario Hezonja\hezonma01,hezonma01,2.0,144.0,2373.0,795.0,322.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [182]:
column_list = list(range(train.shape[1]))
column_list.remove(3)
column_list.remove(4)
column_list.remove(22)
column_list.remove(23)
cols = train.columns[column_list]
train_input = train[cols]
column_list = list(range(test.shape[1]))
column_list.remove(3)
column_list.remove(4)
column_list.remove(22)
column_list.remove(23)
cols = test.columns[column_list]
test_input = test[cols]

In [55]:
fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
train_imputed = pd.DataFrame(fill_NaN.fit_transform(train_input))
train_imputed.columns = train_input.columns
train_imputed.index = train_input.index

fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
test_imputed = pd.DataFrame(fill_NaN.fit_transform(test_input))
test_imputed.columns = test_input.columns
test_imputed.index = test_input.index


In [56]:
rf = RandomForestRegressor(n_estimators=20)
rf.fit(train_imputed, train['NBA VORP'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [57]:
r2 = r2_score(test['NBA VORP'], rf.predict(test_imputed[cols]))
mse = np.mean((test['NBA VORP'] - rf.predict(test_imputed[cols]))**2)

In [58]:
print(r2)
print(mse)

0.991816971319
0.490604906542


In [59]:
def queryYear(df, year):
    df_filtered = df[df['Year'] == year]
    return df_filtered

def preProcessing(df):
#     df_filtered = df.dropna()
#     one_hot_college = pd.get_dummies(df_filtered['Round 1 College'])
#     one_hot_college = one_hot_college.apply(pd.to_numeric, errors='ignore')
#     one_hot_team = pd.get_dummies(df_filtered['Tm'])
#     one_hot_team = one_hot_team.apply(pd.to_numeric, errors='ignore')
#     df_filtered = df_filtered.drop('Round 1 College', axis = 1)
#     df_filtered = df_filtered.drop('Tm', axis = 1)
#     df_filtered = df_filtered.join(one_hot_college)
#     df_filtered = df_filtered.join(one_hot_team)
#     drop_index = []
#    for i in range(0,len(df)):
#        if math.isnan(df_filtered['NBA VORP'][i]) or math.isnan(df_filtered['NBA VORP Count'][i]):
#            drop_index.append(i)    
#   df_filtered = df_filtered.drop(df_filtered.index[drop_index])
    column_list = list(range(df_filtered.shape[1]))
    column_list.remove(3)
    column_list.remove(4)
    column_list.remove(23)
    cols = df_filtered.columns[column_list]
    df_input = df_filtered[cols]
    fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
    df_imputed = pd.DataFrame(fill_NaN.fit_transform(df_input))
    df_imputed.columns = df_input.columns
    df_imputed.index = df_input.index
    return df_imputed

    
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

In [148]:
df_VORP = df['NBA VORP'].groupby([df['Year']]).mean()

In [152]:
df_VORP = df_VORP.to_frame()

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [153]:
print(df_VORP)

      NBA VORP
Year          
1985  1.678788
1986  1.436765
1987  4.161818
1988  2.163793
1989  1.668750
1990  1.759615
1991  2.122727
1992  2.720833
1993  2.311628
1994  3.977778
1995  3.828000
1996  3.417021
1997  3.791489
1998  4.217857
1999  2.406522
2000  1.696000
2001  2.118367
2002  1.312500
2003  4.859574
2004  2.608696
2005  2.792727
2006  1.444231
2007  2.034694
2008  2.531373
2009  2.786000
2010  1.827451
2011  1.975926
2012  1.180357
2013  1.605882
2014  0.446154
2015  0.373810
2016 -0.182222


In [160]:
predictions = []
for i in range(2010,2017):
    df_temp = queryYear(df_query,i)
    predictions.append(np.mean(rf.predict(df_temp[cols])))

In [161]:
print(predictions)

[1.5503333333333336, 1.7761666666666669, 1.0978333333333334, 1.363, 0.38608333333333333, 0.26083333333333336, -0.14108333333333334]


In [155]:
for i in range(0,3):
    df_VORP.iloc[29+i][0] = predictions[i]
    

In [157]:
df_VORP.tail()

,NBA VORP
Year,
2012,1.180357
2013,1.605882
2014,0.386083
2015,0.260833
2016,-0.141083
